<a href="https://colab.research.google.com/github/fanar-barboud/Data-structure/blob/main/Phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

data = pd.read_csv("/content/garment_workshop.csv")

threshold = 0.75
data["label"] = (data["actual_productivity"] >= threshold).astype(int)

X = data.drop(columns=["label", "actual_productivity"])
y = data["label"]

X = pd.get_dummies(X, drop_first=True)


# Classification

# 1) 90% / 10% — Gini

In [ ]:
# Decision Tree - Gini - 90% Train / 10% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.9,
    random_state=1,
    stratify=y
)

model_90_gini = DecisionTreeClassifier(criterion="gini", random_state=1)
model_90_gini.fit(X_train, y_train)

y_pred_90_gini = model_90_gini.predict(X_test)

acc_90_gini = accuracy_score(y_test, y_pred_90_gini)
cm_90_gini = confusion_matrix(y_test, y_pred_90_gini)

print("Gini - 90/10")
print("Accuracy:", acc_90_gini)
print("Confusion Matrix:\n", cm_90_gini)


Gini - 90/10
Accuracy: 0.85
Confusion Matrix:
 [[40  8]
 [10 62]]


# 2) 90% / 10% — Entropy (IG)

In [ ]:
# Decision Tree - Entropy - 90% Train / 10% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.9,
    random_state=1,
    stratify=y
)

model_90_ent = DecisionTreeClassifier(criterion="entropy", random_state=1)
model_90_ent.fit(X_train, y_train)

y_pred_90_ent = model_90_ent.predict(X_test)

acc_90_ent = accuracy_score(y_test, y_pred_90_ent)
cm_90_ent = confusion_matrix(y_test, y_pred_90_ent)

print("Entropy - 90/10")
print("Accuracy:", acc_90_ent)
print("Confusion Matrix:\n", cm_90_ent)


Entropy - 90/10
Accuracy: 0.8416666666666667
Confusion Matrix:
 [[37 11]
 [ 8 64]]


# 3) 80% / 20% — Gini

In [ ]:
# Decision Tree - Gini - 80% Train / 20% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8,
    random_state=1,
    stratify=y
)

model_80_gini = DecisionTreeClassifier(criterion="gini", random_state=1)
model_80_gini.fit(X_train, y_train)

y_pred_80_gini = model_80_gini.predict(X_test)

acc_80_gini = accuracy_score(y_test, y_pred_80_gini)
cm_80_gini = confusion_matrix(y_test, y_pred_80_gini)

print("Gini - 80/20")
print("Accuracy:", acc_80_gini)
print("Confusion Matrix:\n", cm_80_gini)


Gini - 80/20
Accuracy: 0.8083333333333333
Confusion Matrix:
 [[ 76  20]
 [ 26 118]]


# 4) 80% / 20% — Entropy

In [ ]:
# Decision Tree - Entropy - 80% Train / 20% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8,
    random_state=1,
    stratify=y
)

model_80_ent = DecisionTreeClassifier(criterion="entropy", random_state=1)
model_80_ent.fit(X_train, y_train)

y_pred_80_ent = model_80_ent.predict(X_test)

acc_80_ent = accuracy_score(y_test, y_pred_80_ent)
cm_80_ent = confusion_matrix(y_test, y_pred_80_ent)

print("Entropy - 80/20")
print("Accuracy:", acc_80_ent)
print("Confusion Matrix:\n", cm_80_ent)


Entropy - 80/20
Accuracy: 0.8083333333333333
Confusion Matrix:
 [[ 76  20]
 [ 26 118]]


# 5) 70% / 30% — Gini

In [ ]:
# Decision Tree - Gini - 70% Train / 30% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.7,
    random_state=1,
    stratify=y
)

model_70_gini = DecisionTreeClassifier(criterion="gini", random_state=1)
model_70_gini.fit(X_train, y_train)

y_pred_70_gini = model_70_gini.predict(X_test)

acc_70_gini = accuracy_score(y_test, y_pred_70_gini)
cm_70_gini = confusion_matrix(y_test, y_pred_70_gini)

print("Gini - 70/30")
print("Accuracy:", acc_70_gini)
print("Confusion Matrix:\n", cm_70_gini)


Gini - 70/30
Accuracy: 0.8444444444444444
Confusion Matrix:
 [[116  29]
 [ 27 188]]


# 6) 70% / 30% — Entropy

In [ ]:
# Decision Tree - Entropy - 70% Train / 30% Test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.7,
    random_state=1,
    stratify=y
)

model_70_ent = DecisionTreeClassifier(criterion="entropy", random_state=1)
model_70_ent.fit(X_train, y_train)

y_pred_70_ent = model_70_ent.predict(X_test)

acc_70_ent = accuracy_score(y_test, y_pred_70_ent)
cm_70_ent = confusion_matrix(y_test, y_pred_70_ent)

print("Entropy - 70/30")
print("Accuracy:", acc_70_ent)
print("Confusion Matrix:\n", cm_70_ent)


Entropy - 70/30
Accuracy: 0.825
Confusion Matrix:
 [[111  34]
 [ 29 186]]


In [ ]:
# Decision Tree - Gini
...
